In [1]:
import pandas as pd


power_foraward_df = pd.read_csv('Datasets/nba-contract-Power Forward.csv')
center_df = pd.read_csv('Datasets/nba-contracts-Center.csv')
forward_df = pd.read_csv('Datasets/nba-contracts-Forward.csv')
guard_df = pd.read_csv('Datasets/nba-contracts-Guard.csv')
point_guard_df = pd.read_csv('Datasets/nba-contracts-Point Guard.csv')
shooting_guard_df = pd.read_csv('Datasets/nba-contracts-Shooting Guard.csv')
small_forward_df = pd.read_csv('Datasets/nba-contracts-Small Forward.csv')

concat_df = pd.concat([power_foraward_df, center_df, forward_df, guard_df, point_guard_df, shooting_guard_df, small_forward_df])

concat_df.head()

,Rank,Player,Pos,Team\n Signed With,Age\n At Signing,Start,End,Yrs,Value,AAV,Type
0,1,Jayson Tatum,PF,BOS\n BOS,26,2025,2029,5,"$313,933,410","$62,786,682",Designated Veteran Extension
1,2,Giannis Antetokounmpo,PF,MIL\n MIL,26,2021,2025,5,"$228,200,420","$45,640,084",Designated Veteran Extension
2,3,Scottie Barnes,PF,TOR\n TOR,22,2025,2029,5,"$224,237,860","$44,847,572",Designated Rookie Extension
3,4,Zion Williamson,PF,NOP\n NOP,22,2023,2027,5,"$197,230,450","$39,446,090",Designated Rookie Extension
4,5,Pascal Siakam,PF,IND\n IND,30,2024,2027,4,"$188,950,272","$47,237,568",Free Agent


In [2]:
concat_df['Team\n                        Signed With'] = concat_df['Team\n                        Signed With'].str.strip()
concat_df['Age\n                        At Signing'] = concat_df['Age\n                        At Signing'].str.strip()

concat_df = concat_df.rename(columns={'Team\n                        Signed With': 'Team Signed'})
concat_df = concat_df.rename(columns={'Age\n                        At Signing': 'Age At Signing'})

concat_df['Team Signed'] = concat_df['Team Signed'].str.replace('\n', '')

# .split() will split the string into a list of strings
concat_df['Team Signed'] = concat_df['Team Signed'].apply(lambda x: x.split()[0])
concat_df.head()

,Rank,Player,Pos,Team Signed,Age At Signing,Start,End,Yrs,Value,AAV,Type
0,1,Jayson Tatum,PF,BOS,26,2025,2029,5,"$313,933,410","$62,786,682",Designated Veteran Extension
1,2,Giannis Antetokounmpo,PF,MIL,26,2021,2025,5,"$228,200,420","$45,640,084",Designated Veteran Extension
2,3,Scottie Barnes,PF,TOR,22,2025,2029,5,"$224,237,860","$44,847,572",Designated Rookie Extension
3,4,Zion Williamson,PF,NOP,22,2023,2027,5,"$197,230,450","$39,446,090",Designated Rookie Extension
4,5,Pascal Siakam,PF,IND,30,2024,2027,4,"$188,950,272","$47,237,568",Free Agent


In [3]:
# Drop duplicates
concat_df.drop_duplicates(subset=['Player', 'Start'], inplace=True)
concat_df.head()

,Rank,Player,Pos,Team Signed,Age At Signing,Start,End,Yrs,Value,AAV,Type
0,1,Jayson Tatum,PF,BOS,26,2025,2029,5,"$313,933,410","$62,786,682",Designated Veteran Extension
1,2,Giannis Antetokounmpo,PF,MIL,26,2021,2025,5,"$228,200,420","$45,640,084",Designated Veteran Extension
2,3,Scottie Barnes,PF,TOR,22,2025,2029,5,"$224,237,860","$44,847,572",Designated Rookie Extension
3,4,Zion Williamson,PF,NOP,22,2023,2027,5,"$197,230,450","$39,446,090",Designated Rookie Extension
4,5,Pascal Siakam,PF,IND,30,2024,2027,4,"$188,950,272","$47,237,568",Free Agent


In [4]:
# Drop rows with Start year before 2000
concat_df = concat_df[concat_df['Start'] >= 2000]
concat_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2379 entries, 0 to 499
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Rank            2379 non-null   object
 1   Player          2379 non-null   object
 2   Pos             2379 non-null   object
 3   Team Signed     2379 non-null   object
 4   Age At Signing  2353 non-null   object
 5   Start           2379 non-null   int64 
 6   End             2379 non-null   int64 
 7   Yrs             2379 non-null   int64 
 8   Value           2379 non-null   object
 9   AAV             2379 non-null   object
 10  Type            1599 non-null   object
dtypes: int64(3), object(8)
memory usage: 223.0+ KB


In [5]:
import pandas as pd

# CPI data from 2000 to 2024 (2024 is hypothetical and should be updated with actual data when available)
cpi_data = {
    'Year': list(range(2000, 2025)),  # Ensure we include 2024
    'CPI': [
        172.2, 177.1, 179.9, 184.0, 188.9, 195.3, 201.6, 207.3, 215.3, 214.5, 
        218.1, 224.9, 229.6, 232.9, 236.7, 237.0, 240.0, 245.1, 251.1, 255.7, 
        258.8, 271.7, 287.5, 299.2, 304.7  # Hypothetical value for 2024
    ]
}

# Convert to DataFrame
cpi_df = pd.DataFrame(cpi_data)

# Function to adjust for inflation
def adjust_for_inflation(value, start_year, cpi_df):
    value = float(value.replace('$', '').replace(',', ''))  # Remove $ and commas
    if start_year > 2024:
        return f"${value:,.2f} "
    if start_year not in cpi_df['Year'].values:
        raise ValueError(f"CPI data for the year {start_year} not found.")
    cpi_start_year = cpi_df.loc[cpi_df['Year'] == start_year, 'CPI'].values[0]
    cpi_target_year = cpi_df.loc[cpi_df['Year'] == 2024, 'CPI'].values[0]  # Adjust to 2024
    adjustment_factor = cpi_target_year / cpi_start_year
    adjusted_value = value * adjustment_factor
    return f"${adjusted_value:,.2f}"

# Adjust 'Value' for inflation based on 'Start' year
concat_df['Value Adjusted'] = concat_df.apply(lambda row: adjust_for_inflation(row['Value'], row['Start'], cpi_df), axis=1)


# Display the cleaned DataFrame
concat_df.head()


,Rank,Player,Pos,Team Signed,Age At Signing,Start,End,Yrs,Value,AAV,Type,Value Adjusted
0,1,Jayson Tatum,PF,BOS,26,2025,2029,5,"$313,933,410","$62,786,682",Designated Veteran Extension,"$313,933,410.00"
1,2,Giannis Antetokounmpo,PF,MIL,26,2021,2025,5,"$228,200,420","$45,640,084",Designated Veteran Extension,"$255,917,070.20"
2,3,Scottie Barnes,PF,TOR,22,2025,2029,5,"$224,237,860","$44,847,572",Designated Rookie Extension,"$224,237,860.00"
3,4,Zion Williamson,PF,NOP,22,2023,2027,5,"$197,230,450","$39,446,090",Designated Rookie Extension,"$200,856,009.74"
4,5,Pascal Siakam,PF,IND,30,2024,2027,4,"$188,950,272","$47,237,568",Free Agent,"$188,950,272.00"


In [6]:
# open the nba_player_stats_2000_2024.csv file
player_stats_df = pd.read_csv('Datasets/nba_player_stats_2000_2024.csv')


In [7]:
# Identify unique values in 'Age At Signing'
concat_df['Age At Signing'].unique()

# Manually adjusted the outliers ages  to the correct values

array(['26', '22', '30', '27', '28', '29', '23', '33', '25', '39', '31',
       '37', '35', '24', '32', '19', '18', '2,007', '20', '21', '2,003',
       '2,000', '2,002', '34', '17', '38', '36', nan, '2,006', '2,004',
       '2,008', '1,999', '13', '15', '2,005'], dtype=object)

In [8]:
contracts_df = pd.read_csv('nba-contracts-inflation-adjusted.csv')

In [9]:
contracts_df['Type'].unique()

array(['Designated Veteran Extension', 'Designated Rookie Extension',
       'Free Agent', 'Veteran Extension', 'maximum',
       'Maximum Veteran Extension', 'rookie-maximum-extension',
       'Rookie Extension', nan, 'sign-and-trade', 'Extension', 'amnesty',
       'rest-of-season', 'extend-and-trade', 'earned', 'Qualifying Offer'],
      dtype=object)

In [10]:
contracts_df['AAV adjusted'] = contracts_df['Value Adjusted'].str.replace('$', '').str.replace(',', '').astype(float) / contracts_df['Yrs']

contracts_df['AAV adjusted'] = contracts_df['AAV adjusted'].apply(lambda x: f"${x:,.2f}")
contracts_df['AAV adjusted']

0       $62,786,682.00
1       $51,183,414.04
2       $44,847,572.00
3       $40,171,201.95
4       $47,237,568.00
             ...      
1660     $4,409,551.37
1661     $2,032,236.55
1662     $4,064,473.10
1663     $4,839,068.29
1664     $1,744,653.57
Name: AAV adjusted, Length: 1665, dtype: object

In [11]:
contracts_df['AAV adjusted'].sort_values(ascending=False)

451     $9,982,516.47
1088    $9,953,179.44
1058    $9,947,797.91
1056    $9,946,973.71
139     $9,945,328.44
            ...      
1659    $1,397,065.57
1643    $1,388,934.88
1371    $1,362,122.96
1656    $1,249,940.23
1361    $1,125,754.97
Name: AAV adjusted, Length: 1665, dtype: object

In [12]:
import pandas as pd

# Convert 'AAV adjusted' column to numeric type
contracts_df['AAV adjusted'] = pd.to_numeric(contracts_df['AAV adjusted'].str.replace('$', '').str.replace(',', ''))


contracts_df['AAV adjusted']

0       62786682.00
1       51183414.04
2       44847572.00
3       40171201.95
4       47237568.00
           ...     
1660     4409551.37
1661     2032236.55
1662     4064473.10
1663     4839068.29
1664     1744653.57
Name: AAV adjusted, Length: 1665, dtype: float64

In [13]:
contracts_df.to_csv('nba-contracts-inflation-adjusted.csv', index=False)

In [14]:
# Add in all useful parameters to the nba-contracts-inflation-adjusted.csv file for analysis

# Calculate the efficiency of each player
player_stats_df['Efficiency'] = (player_stats_df['PTS'] + player_stats_df['REB'] + player_stats_df['AST'] + player_stats_df['STL'] + player_stats_df['BLK']) - (player_stats_df['FGA'] - player_stats_df['FGM']) - (player_stats_df['FTA'] - player_stats_df['FTM']) - player_stats_df['TOV']

# Calculate True Shooting Percentage (TS%)
player_stats_df['TS%'] = player_stats_df['PTS'] / (2 * (player_stats_df['FGA'] + 0.44 * player_stats_df['FTA']))

# Calculate Player Impact Estimate (PIE)
player_stats_df['PIE'] = (player_stats_df['PTS'] + player_stats_df['FGM'] + player_stats_df['FTM'] - player_stats_df['FGA'] - player_stats_df['FTA'] + player_stats_df['OREB'] + player_stats_df['DREB'] + player_stats_df['AST'] + player_stats_df['STL'] + player_stats_df['BLK'] - player_stats_df['TOV']) / player_stats_df[['PTS', 'FGM', 'FTM', 'FGA', 'FTA', 'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TOV']].sum().sum()


player_stats_df.head()

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,Season,Efficiency,TS%,PIE
0,920,A.C. Green,A.C.,1610612748,MIA,37.0,82,50,32,0.610,...,291,110,286,126,26,291,2000-01,6.2,0.487435,0.000018
1,2062,A.J. Guyton,A.J.,1610612741,CHI,23.0,33,6,27,0.182,...,223,413,282,224,26,272,2000-01,5.4,0.498339,0.000016
2,243,Aaron McKie,Aaron,1610612755,PHI,28.0,76,51,25,0.671,...,99,24,93,77,4,86,2000-01,13.9,0.550076,0.000041
3,1425,Aaron Williams,Aaron,1610612751,NJN,29.0,82,26,56,0.317,...,115,418,97,53,26,100,2000-01,13.9,0.532804,0.000041
4,228,Adam Keefe,Adam,1610612744,GSW,31.0,67,14,53,0.209,...,365,358,320,224,26,331,2000-01,4.5,0.447067,0.000013


In [15]:
from sklearn.preprocessing import MinMaxScaler

# Scale the 'PIE' column to a 0-100 scale
scaler = MinMaxScaler(feature_range=(0, 100))
player_stats_df['PIE_scaled'] = scaler.fit_transform(player_stats_df[['PIE']])
player_stats_df['PIE_scaled']

0        23.831776
1        21.962617
2        41.822430
3        41.822430
4        19.859813
           ...    
11654    53.037383
11655    25.700935
11656    20.327103
11657    28.037383
11658    65.186916
Name: PIE_scaled, Length: 11659, dtype: float64

In [16]:
# Map the contract signed season to the corresponding previous season

season_mapping = {
    2026: '2023-24',
    2025: '2023-24',
    2024: '2023-24',
    2023: '2022-23',
    2022: '2021-22',
    2021: '2020-21',
    2020: '2019-20',
    2019: '2018-19',
    2018: '2017-18',
    2017: '2016-17',
    2016: '2015-16',
    2015: '2014-15',
    2014: '2013-14',
    2013: '2012-13',
    2012: '2011-12',
    2011: '2010-11',
    2010: '2009-10',
    2009: '2008-09',
    2008: '2007-08',
    2007: '2006-07',
    2006: '2005-06',
    2005: '2004-05',
    2004: '2003-04',
    2003: '2002-03',
    2002: '2001-02',
    2001: '2000-01',
    2000: '1999-00'
}


# Add a "Performance Season" column to contracts_df based on the mapping
contracts_df['Performance Season'] = contracts_df['Start'].map(season_mapping)

contracts_df = contracts_df[contracts_df['Type'] != 'Rookie']

contracts_df.info()

player_stats_df.to_csv('nba_player_stats_2000_2024.csv', index=False)
contracts_df.to_csv('nba-contracts-inflation-adjusted.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1665 entries, 0 to 1664
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rank                1665 non-null   object 
 1   Player              1665 non-null   object 
 2   Pos                 1665 non-null   object 
 3   Team Signed         1665 non-null   object 
 4   Age At Signing      1665 non-null   int64  
 5   Start               1665 non-null   int64  
 6   End                 1665 non-null   int64  
 7   Yrs                 1665 non-null   int64  
 8   Value               1665 non-null   object 
 9   AAV                 1665 non-null   object 
 10  Type                885 non-null    object 
 11  Value Adjusted      1665 non-null   object 
 12  AAV adjusted        1665 non-null   float64
 13  Performance Season  1665 non-null   object 
dtypes: float64(1), int64(4), object(9)
memory usage: 182.2+ KB


In [17]:
nan_rows = contracts_df[contracts_df['Performance Season'].isnull()]
nan_rows

,Rank,Player,Pos,Team Signed,Age At Signing,Start,End,Yrs,Value,AAV,Type,Value Adjusted,AAV adjusted,Performance Season


In [18]:
# Merge the contracts_df and player_stats_df DataFrames
from rapidfuzz import process, fuzz

#merged_df = contracts_df.merge(player_stats_df, left_on=['Player', 'Performance Season'], right_on=['PLAYER_NAME', 'Season'], how='left')

# Function to match similar names using rapidfuzz
def match_names(name, names_list, threshold=70):
    match, score, _ = process.extractOne(name, names_list, scorer=fuzz.token_sort_ratio)
    if score >= threshold and score < 100:
        print(f"Matched '{name}' to '{match}' with a score of {score}")
        return match
    return None

# Apply fuzzy matching for the 'Player' column in contracts_df,   Match all the fuzzy names and retai the oriignal names
player_stats_df_names = player_stats_df['PLAYER_NAME'].tolist()
contracts_df['Matched_Player'] = contracts_df['Player'].apply(lambda x: match_names(x, player_stats_df_names)).fillna(contracts_df['Player'])

# Merge DataFrames using the matched player names
merged_df = contracts_df.merge(player_stats_df, left_on=['Matched_Player', 'Performance Season'], right_on=['PLAYER_NAME', 'Season'], how='left')


# Drop the 'Matched_Player' column as it's no longer needed
merged_df = merged_df.drop(columns=['Matched_Player'])

Matched 'Davis Bertans' to 'Dairis Bertans' with a score of 88.88888888888889
Matched 'PJ Washington' to 'P.J. Washington' with a score of 92.85714285714286
Matched 'Luc Richard Mbah a Moute' to 'Luc Mbah a Moute' with a score of 80.0
Matched 'J.J Hickson' to 'JJ Hickson' with a score of 95.23809523809523
Matched 'Davis Bertans' to 'Dairis Bertans' with a score of 88.88888888888889
Matched 'G.G. Jackson' to 'GG Jackson II' with a score of 80.0
Matched 'LaVoy Allen' to 'Lavoy Allen' with a score of 90.9090909090909
Matched 'LaVoy Allen' to 'Lavoy Allen' with a score of 90.9090909090909
Matched 'Jamorio Moon' to 'Jamario Moon' with a score of 91.66666666666666
Matched 'Nicolas Claxton' to 'Nic Claxton' with a score of 84.61538461538461
Matched 'Omer Asik' to 'Ömer Aşik' with a score of 77.77777777777779
Matched 'Omer Asik' to 'Ömer Aşik' with a score of 77.77777777777779
Matched 'Mohamed Bamba' to 'Mo Bamba' with a score of 76.19047619047619
Matched 'Nicolas Claxton' to 'Nic Claxton' wit

In [19]:
merged_df.dropna(subset=['TS%'], inplace=True)

In [20]:
df = pd.read_csv('final.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1543 entries, 0 to 1542
Data columns (total 85 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rank                   1543 non-null   object 
 1   Player                 1543 non-null   object 
 2   Pos                    1543 non-null   object 
 3   Team Signed            1543 non-null   object 
 4   Age At Signing         1543 non-null   int64  
 5   Start                  1543 non-null   int64  
 6   End                    1543 non-null   int64  
 7   Yrs                    1543 non-null   int64  
 8   Value                  1543 non-null   object 
 9   AAV                    1543 non-null   object 
 10  Type                   831 non-null    object 
 11  Value Adjusted         1543 non-null   int64  
 12  AAV adjusted           1543 non-null   int64  
 13  Performance Season     1543 non-null   object 
 14  PLAYER_ID              1543 non-null   float64
 15  PLAY

In [21]:
merged_df['Value Adjusted'] = merged_df['Value Adjusted'].str.replace('$', '').str.replace(',', '').astype(float).astype(int)
merged_df['AAV adjusted'] = merged_df['AAV adjusted'].astype(float).astype(int)

In [22]:
merged_df['Value Adjusted'].describe()
merged_df.to_csv('final.csv', index=False)

In [23]:
merged_df.head()

,Rank,Player,Pos,Team Signed,Age At Signing,Start,End,Yrs,Value,AAV,...,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,Season,Efficiency,TS%,PIE,PIE_scaled
0,1,Jayson Tatum,PF,BOS,26,2025,2029,5,"$313,933,410","$62,786,682",...,3.0,10.0,22.0,38.0,9.0,2023-24,27.7,0.604549,0.000081,74.065421
1,2,Giannis Antetokounmpo,PF,MIL,26,2021,2025,5,"$228,200,420","$45,640,084",...,10.0,3.0,8.0,6.0,2.0,2020-21,33.3,0.633454,0.000098,87.149533
2,3,Scottie Barnes,PF,TOR,22,2025,2029,5,"$224,237,860","$44,847,572",...,432.0,19.0,22.0,7.0,25.0,2023-24,25.1,0.567016,0.000074,68.224299
3,4,Zion Williamson,PF,NOP,22,2023,2027,5,"$197,230,450","$39,446,090",...,17.0,23.0,105.0,39.0,28.0,2022-23,27.0,0.650520,0.000079,72.429907
4,5,Pascal Siakam,PF,IND,30,2024,2027,4,"$188,950,272","$47,237,568",...,147.0,45.0,45.0,16.0,48.0,2023-24,23.6,0.599448,0.000069,64.252336


In [27]:
player_stats_df[player_stats_df['PLAYER_NAME'] == 'Andre Drummond']

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,Season,Efficiency,TS%,PIE,PIE_scaled
5414,203083,Andre Drummond,Andre,1610612765,DET,19.0,60,23,37,0.383,...,250,108,73,27,126,2012-13,13.7,0.573461,0.000040,41.355140
5887,203083,Andre Drummond,Andre,1610612765,DET,20.0,81,29,52,0.358,...,406,23,2,27,34,2013-14,22.6,0.599467,0.000066,62.149533
6369,203083,Andre Drummond,Andre,1610612765,DET,21.0,82,32,50,0.390,...,266,17,2,21,28,2014-15,20.8,0.504386,0.000061,57.710280
6860,203083,Andre Drummond,Andre,1610612765,DET,22.0,81,43,38,0.531,...,89,10,1,25,16,2015-16,21.9,0.497910,0.000064,60.514019
7337,203083,Andre Drummond,Andre,1610612765,DET,23.0,81,36,45,0.444,...,438,29,8,24,41,2016-17,21.3,0.517661,0.000063,59.112150
7823,203083,Andre Drummond,Andre,1610612765,DET,24.0,78,37,41,0.474,...,291,8,2,32,14,2017-18,27.2,0.553751,0.000080,72.897196
8361,203083,Andre Drummond,Andre,1610612765,DET,25.0,79,40,39,0.506,...,82,10,1,38,20,2018-19,27.2,0.554914,0.000080,72.897196
8891,203083,Andre Drummond,Andre,1610612739,CLE,26.0,57,21,36,0.368,...,452,11,5,29,14,2019-20,26.8,0.550099,0.000079,71.962617
9420,203083,Andre Drummond,Andre,1610612747,LAL,27.0,46,19,27,0.413,...,416,44,16,29,57,2020-21,20.6,0.518081,0.000061,57.476636
9962,203083,Andre Drummond,Andre,1610612751,BKN,28.0,73,42,31,0.575,...,220,113,38,40,140,2021-22,15.9,0.575131,0.000047,46.495327
